# Driver Safety Detector 

### The following program performs a safety detection applications for the driver
#### The program detection consists of :
#####  1. Yawn Detection : In the 5 minutes of span if the driver yawns 3 or more times then the driver tiredness is alerted with the help of an alarm  
#####  2.  Dizziness Detection: If the driver is feeling dizzy then an alarm is triggered                                                                                                             
#####  3.  Phone usage detection : If the driver tries to use a cell phone while driving then  an alarm is triggered

### Importing Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib 
import time
import playsound

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

## Yawn Detector

In [ ]:
capture = cv2.VideoCapture(0)      
is_first = True
first_time = 0.0
counter = 0
prev_time = 0.0
alert_alarm = False

while True:      
    ret,frame = capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:

        
        landmarks = predictor(gray,face)
        
        ul_y1_top = landmarks.part(50).y
        ll_y1_bottom = landmarks.part(58).y
        ul_y2_top = landmarks.part(52).y
        ll_y2_bottom = landmarks.part(56).y
                
        if abs(ul_y1_top-ll_y1_bottom) and abs(ul_y2_top - ll_y2_bottom) > 34:
            if is_first:
                first_time = time.time()
                is_first = False
            elif is_first == False and  (time.time() - first_time) < 300 and (time.time() - prev_time) > 7:
                prev_time = time.time()
                counter += 1
                
                if counter == 3:
                    is_first = True
                    first_time = 0.0
                    counter = 0
                    prev_time = 0.0
                    alert_alarm = True
                    print('Alert')
                    
        
    if alert_alarm == True:
        alert_alarm = False
        playsound.playsound('dataset/Alarm Alert Effect-SoundBible.com-462520910-[AudioTrimmer.com].mp3')
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
capture.release() 
cv2.destroyAllWindows()

# Dizziness Detector

In [ ]:
capture = cv2.VideoCapture(0)      
status = True
first_blink = 0.0
alert_alarm = False

while True:      
    
    ret,frame = capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
        landmarks = predictor(gray,face)
        
        le_y1_top = landmarks.part(37).y
        le_y1_bottom = landmarks.part(41).y
        le_y2_top = landmarks.part(38).y
        le_y2_bottom = landmarks.part(40).y
        
        re_y1_top = landmarks.part(43).y
        re_y1_bottom = landmarks.part(47).y
        re_y2_top = landmarks.part(44).y
        re_y2_bottom = landmarks.part(46).y
        

        if abs(le_y1_top-le_y1_bottom) and abs(le_y2_top-le_y2_bottom) and abs(re_y1_top-re_y1_bottom) and abs(re_y2_top-re_y2_bottom) < 5:
            
            if status == True:
                first_blink = time.time()
                status = False
            
            elif status == False and time.time() - first_blink > 4:
                first_blink = 0.0
                status = True
                print('Alert')
                
    
    
    if alert_alarm == True:
        alert_alarm = False
        playsound.playsound('dataset/Alarm Alert Effect-SoundBible.com-462520910-[AudioTrimmer.com].mp3')
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
capture.release() 
cv2.destroyAllWindows()

## Cell Phone Usage Detector

In [ ]:
net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')

In [ ]:
classes = []
with open('coco.names','r') as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1 ] for i in net.getUnconnectedOutLayers()]

In [ ]:
capture = cv2.VideoCapture(0)

while True:
    _, frame = capture.read()
    
    
    height,width,channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    


    boxes = []
    confidences = []
    class_ids = []

    

    for out in outs:
        for detection in out:
            score = detection[5:]
            class_id = np.argmax(score)
            confidence = score[class_id]
            if confidence > 0.3:
                centre_x = int(detection[0] * width)
                centre_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(centre_x - w/2)
                y = int(centre_y - h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)


    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.3,0.3)

    
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            if label == 'cell phone':
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                playsound.playsound('dataset/Alarm Alert Effect-SoundBible.com-462520910-[AudioTrimmer.com].mp3')
            
    cv2.imshow('Image',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    
capture.release()
cv2.destroyAllWindows()